In [1]:
import numpy as np
from vlmc import VLMC, BICSolver

import os
import re
import glob
import copy
glob.glob('./*')

['./OrderEst_ex3',
 './OrderEst_ex4',
 './examples.ipynb',
 './examples_BIC.ipynb',
 './OrderEst_ex2',
 './vlmc',
 './OrderEst_ex1']

In [2]:
# Set random seed
SEED=4007
np.random.seed(SEED)

# Number of parallel jobs for executions
NJOBS=os.cpu_count()

In [3]:
def parse_txt_input(input_path):
    """
        Função para ler e decodificar arquivos txt. Lê-se o arquivo e analisa-se cada linha
        para obter o respectivo valor do estado. Retorna-se um array com a sequência de estados observados (observed_sequence).
    """

    observed_sequence = []
    with open(
        input_path,
        mode='r',
        encoding='utf-8'
    ) as f:
        lines = f.readlines()
        # Use regular expression to obtain sequence from input
        observed_sequence = [
            int(i) for i in re.search('[0-9]{1000,100000}', lines[-1]).group(0)
        ]
        
    return observed_sequence

# BIC

### 1st Order sample

In [4]:
###################
# GENERATE SAMPLE #
###################

# Length of generated sample
N_SAMPLES = 10000

# Declare state space and mapping from state number to zero-indexed position in transition matrix
STATE_SPACE = [
    0,
    1
]
STATE_TO_INDEX_MAP = {
    s: i for i, s in enumerate(STATE_SPACE)
}

# Transition probabilities
Q = np.array([
    [0.2, 0.8],
    [0.5, 0.5]
])

# Initial state
X = [1, 1]

# 
for i in range(len(X)-1, N_SAMPLES-1):
    current_s = X[i]
    X.append(
        np.random.choice(
            a=STATE_SPACE,
            p=Q[current_s,:]
        )
    )
    
print('-'*100)
print('Length of generated sample: {}'.format(len(X)))
print('First 10 symbols of generated sample: {}'.format(X[:10]))
print('-'*100)

----------------------------------------------------------------------------------------------------
Length of generated sample: 10000
First 10 symbols of generated sample: [1, 1, 0, 1, 1, 1, 1, 1, 0, 1]
----------------------------------------------------------------------------------------------------


In [5]:
%%time
# Instantiate VLMC object
chain = VLMC(
    vocabulary=STATE_SPACE,
    max_order=8
)

# Fit chain to sample using BIC
chain.fit(
    X=X,
    method='bic',
    njobs=NJOBS
)

CPU times: user 150 ms, sys: 67.7 ms, total: 218 ms
Wall time: 463 ms


In [6]:
# Get diagram of estimated tree
chain.show_tree()

root
├── 0
└── 1



In [7]:
# Transition probabilities for leaves
leaves = chain.get_leaves()
{n.word: n.transition_probabilities for n in leaves}

{'0': {'0': 0.19979161239906226, '1': 0.8004688721021099},
 '1': {'0': 0.49943117178612056, '1': 0.5015439622948156}}

In [8]:
# Ground truth for leaves probabilities
Q[0], Q[1]

(array([0.2, 0.8]), array([0.5, 0.5]))

### 2nd Order sample

In [9]:
###################
# GENERATE SAMPLE #
###################

# Length of generated sample
N_SAMPLES = 10000

# Declare state space and mapping from state number to zero-indexed position in transition matrix
STATE_SPACE = [
    0,
    1
]
STATE_TO_INDEX_MAP = {
    s: i for i, s in enumerate(STATE_SPACE)
}

Q = np.array([
    [[0.2, 0.8],
    [0.1, 0.9]],
    [[0.4, 0.6],
    [0.8, 0.2]]
])

X = [1, 1]
for i in range(len(X)-1, N_SAMPLES-1):
    current_s = X[i-1:i+1]
    X.append(
        np.random.choice(
            a=STATE_SPACE,
            p=Q[tuple(current_s)]
        )
    )

print('-'*100)
print('Length of generated sample: {}'.format(len(X)))
print('First 10 symbols of generated sample: {}'.format(X[:10]))
print('-'*100)

----------------------------------------------------------------------------------------------------
Length of generated sample: 10000
First 10 symbols of generated sample: [1, 1, 0, 1, 1, 0, 1, 1, 1, 0]
----------------------------------------------------------------------------------------------------


In [10]:
%%time
# Instantiate VLMC object
chain = VLMC(
    vocabulary=STATE_SPACE,
    max_order=8
)

# Fit chain to sample using BIC
chain.fit(
    X=X,
    method='bic',
    njobs=NJOBS
)

CPU times: user 132 ms, sys: 62.2 ms, total: 194 ms
Wall time: 440 ms


In [11]:
# Get diagram of estimated tree
chain.show_tree()

root
├── 0
│   ├── 00
│   └── 10
└── 1
    ├── 01
    └── 11



In [12]:
# Transition probabilities for leaves
leaves = chain.get_leaves()
{n.word: n.transition_probabilities for n in leaves}

{'00': {'0': 0.1836734693877551, '1': 0.8163265306122449},
 '10': {'0': 0.3901734104046243, '1': 0.6105491329479769},
 '01': {'0': 0.09104046242774566, '1': 0.9096820809248555},
 '11': {'0': 0.8040217044366422, '1': 0.1966166613469518}}

In [13]:
# Ground truth for transition probabilities
Q[0,0], Q[1,0], Q[0,1], Q[1, 1]

(array([0.2, 0.8]), array([0.4, 0.6]), array([0.1, 0.9]), array([0.8, 0.2]))

### 4th order complex tree

In [14]:
def dirichlet_gen_fn(vocabulary_size):
    """
        This function generates a probability distribution according to an even
        dirichlet. Extreme values for probabilities are avoided.
    """
    is_valid=False
    while not is_valid:
        # Generate sample
        probs = np.random.dirichlet(
            alpha = [0.5]*vocabulary_size
        )
        # Check if sample contains extreme probability values
        if max(probs)<0.8 and min(probs)>0.2:
            is_valid=True
            
    return probs

In [15]:
###################
# GENERATE SAMPLE #
###################

# Length of generated sample
N_SAMPLES=100000
MAX_DEPTH = 4
STATE_SPACE = [
    0,
    1,
    2
]

# Contexts in tree
contexts = [
    '0',
    '11',
    '21',
    '001',
    '101',
    '0201',
    '1201',
    '2201',
    '02',
    '12',
    '022',
    '122',
    '222'
]

# Obtain random transition probabilities for contexts
context_probs = {
    c: dirichlet_gen_fn(
        vocabulary_size=len(STATE_SPACE)
    ) for c in contexts
}

context_probs

{'0': array([0.41951535, 0.25155903, 0.32892562]),
 '11': array([0.36489771, 0.34320296, 0.29189932]),
 '21': array([0.4564581 , 0.24024402, 0.30329788]),
 '001': array([0.21422521, 0.35177914, 0.43399565]),
 '101': array([0.20889002, 0.45086866, 0.34024133]),
 '0201': array([0.20557789, 0.33670726, 0.45771484]),
 '1201': array([0.21845778, 0.46842033, 0.31312189]),
 '2201': array([0.52577593, 0.27250313, 0.20172094]),
 '02': array([0.27607134, 0.47306216, 0.2508665 ]),
 '12': array([0.2266841 , 0.49953721, 0.27377869]),
 '022': array([0.26165118, 0.37433456, 0.36401426]),
 '122': array([0.205716  , 0.20313505, 0.59114895]),
 '222': array([0.37803698, 0.37250095, 0.24946207])}

In [16]:
def context_identification_fn(full_word, ctxts):
    """
        This function finds the context of full_word. Initially, full_word will always have
        length equal to the maximum depth of theoretical tree. Context matching is performed
        by iteratively reducing size of analyzed past in full_word as to eventually match a
        a context in ctxts.
    """
    context_found = False
    context=None
    word = copy.deepcopy(full_word)
    # Reduce size of analyzed past in word iteratively
    # until a context is matched
    while not context_found:
        if len(word)==0:
            import pdb; pdb.set_trace()
        # Check if word is context
        if word in ctxts:
            context=word
            context_found=True
        # If not, analyze one less step into the past
        else:
            word = word[1:]
            
            
    return context

In [17]:
X = [0, 0, 1, 1]
i=MAX_DEPTH
# Iteratively window X to generate next context
while len(X) < N_SAMPLES:
    # Get window of length MAX_DEPTH
    word = ''.join(
        [str(x) for x in X[i-MAX_DEPTH:i]]
    )
    # Associate word to context
    ctxt = context_identification_fn(
        full_word=word,
        ctxts=contexts
    )
    # Sample from transition distribution to obtain
    # next symbol in sample
    new_state = np.random.choice(
        a=STATE_SPACE,
        p=context_probs[ctxt]
    )
    
    # Save and update
    X.append(new_state)
    i+=1

print('-'*100)
print('Length of generated sample: {}'.format(len(X)))
print('First 10 symbols of generated sample: {}'.format(X[:10]))
print('-'*100)

----------------------------------------------------------------------------------------------------
Length of generated sample: 100000
First 10 symbols of generated sample: [0, 0, 1, 1, 1, 0, 2, 2, 1, 1]
----------------------------------------------------------------------------------------------------


In [18]:
%%time
# Instantiate VLMC object
chain = VLMC(
    vocabulary=STATE_SPACE,
    max_order=8
)

# Fit chain to sample using BIC
chain.fit(
    X=X,
    method='bic',
    njobs=NJOBS
)

CPU times: user 1.35 s, sys: 191 ms, total: 1.54 s
Wall time: 1min 45s


In [19]:
# Get diagram of estimated tree
chain.show_tree()

root
├── 0
├── 1
│   ├── 01
│   │   ├── 001
│   │   ├── 101
│   │   └── 201
│   │       ├── 0201
│   │       ├── 1201
│   │       └── 2201
│   ├── 11
│   └── 21
└── 2
    ├── 02
    ├── 12
    └── 22
        ├── 022
        ├── 122
        └── 222



In [20]:
# Transition probabilities for leaves
leaves = chain.get_leaves()
{n.word: n.transition_probabilities for n in leaves}

{'0': {'0': 0.42177961572003264,
  '1': 0.2509353813261316,
  '2': 0.32736939826145667},
 '001': {'0': 0.2046875, '1': 0.34817708333333336, '2': 0.4473958333333333},
 '101': {'0': 0.20548862115127176,
  '1': 0.4605087014725569,
  '2': 0.33400267737617134},
 '0201': {'0': 0.19469026548672566,
  '1': 0.3223767383059418,
  '2': 0.4829329962073325},
 '1201': {'0': 0.22631578947368422,
  '1': 0.45964912280701753,
  '2': 0.3140350877192982},
 '2201': {'0': 0.5041095890410959, '1': 0.2958904109589041, '2': 0.2},
 '11': {'0': 0.36828694629557035,
  '1': 0.33918071344570755,
  '2': 0.29272834182673463},
 '21': {'0': 0.46384750219106047,
  '1': 0.24094361671048786,
  '2': 0.29520888109845167},
 '02': {'0': 0.2811962873839807,
  '1': 0.4725850807837745,
  '2': 0.24630457201787556},
 '12': {'0': 0.22407194795254498,
  '1': 0.49464217374665137,
  '2': 0.28128587830080365},
 '022': {'0': 0.2655268667131891,
  '1': 0.374738311235171,
  '2': 0.3597348220516399},
 '122': {'0': 0.18435374149659864,
  '1

In [21]:
# Ground truth for transition probabilities
context_probs

{'0': array([0.41951535, 0.25155903, 0.32892562]),
 '11': array([0.36489771, 0.34320296, 0.29189932]),
 '21': array([0.4564581 , 0.24024402, 0.30329788]),
 '001': array([0.21422521, 0.35177914, 0.43399565]),
 '101': array([0.20889002, 0.45086866, 0.34024133]),
 '0201': array([0.20557789, 0.33670726, 0.45771484]),
 '1201': array([0.21845778, 0.46842033, 0.31312189]),
 '2201': array([0.52577593, 0.27250313, 0.20172094]),
 '02': array([0.27607134, 0.47306216, 0.2508665 ]),
 '12': array([0.2266841 , 0.49953721, 0.27377869]),
 '022': array([0.26165118, 0.37433456, 0.36401426]),
 '122': array([0.205716  , 0.20313505, 0.59114895]),
 '222': array([0.37803698, 0.37250095, 0.24946207])}